In [1]:
import os
import cv2
import face_recognition
import glob 
import numpy as np
import time
import pyttsx3 as tts
from scipy.spatial import distance as dist

In [2]:
speaker = tts.init()

In [3]:


def swift_speak(audio_string):
    print(f'Swift: {audio_string}')
    voices = speaker.getProperty('voices')
    rate = speaker.getProperty('rate')
    speaker.setProperty('rate', 150)
    speaker.setProperty('voice', voices[1].id)
    speaker.say(audio_string)
    speaker.runAndWait()

In [4]:
img_path = 'images'   

In [5]:
known_face_encodings = []
known_face_names = []

def load_encoding_images(images_path):
  
        # Store image encoding and names
    for img_path in glob.glob(images_path +'/*'):
        img = cv2.imread(img_path)
        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        name = os.path.splitext(os.path.basename(img_path))[0]
          
        img_encoding = face_recognition.face_encodings(rgb_img)[0]

            # Store file name and file encoding
        known_face_encodings.append(img_encoding)
        known_face_names.append(name)
    print("Encoding images loaded")


In [6]:
load_encoding_images(img_path)

Encoding images loaded


In [17]:
def recognize(img, ear):
    face_locations = []
    face_encodings = []
    face_names = []
    
    if ear < 0.15:
        cv2.putText(frame, 'You blinked', (10, 30), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 255), 2)
        
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(img)
        face_encodings = face_recognition.face_encodings(img, face_locations)
        
        # face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"
            
             # Use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)
    else:
        cv2.putText(frame, 'No Liveliness Detected', (10, 30), cv2.FONT_HERSHEY_COMPLEX, 1.0, (255, 255, 255, 255), 1)
    
   
    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4
        
        # Add text to speech
        # swift_speak(f'Welcome, {name}')
        
        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (255, 255, 255, 255), 1)
        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 30), (right, bottom), (0, 0, 0), cv2.FILLED)
        cv2.putText(frame, name, (left+6, bottom - 6), cv2.FONT_HERSHEY_DUPLEX, 1.0, (255, 255, 255), 1)


In [12]:
def get_ear(eye):

    # compute the euclidean distances between the two sets of
    # vertical eye landmarks (x, y)-coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])

    # compute the euclidean distance between the horizontal
    # eye landmark (x, y)-coordinates
    C = dist.euclidean(eye[0], eye[3])

    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)

    # return the eye aspect ratio
    return ear

In [18]:
cam = cv2.VideoCapture(0)
cTime = 0
pTime = 0
    
while True:
    ret, frame = cam.read()
    if not ret:
        break
    
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    small_rgb = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
    
    landmarks = face_recognition.face_landmarks(small_rgb)
    
    
    cTime = time.time()
    fps = 1 / (cTime-pTime)
    pTime = cTime
    
    for landmark in landmarks:
        leftEye = landmark['left_eye']
        rightEye = landmark['right_eye']
        
        # Get aspect ratios for left and right eyes
        leftEar = get_ear(leftEye)
        rightEar = get_ear(rightEye)
        
        ear = (leftEar + rightEar) / 2.0

        # recognise
        recognize(small_rgb, ear)

        # if ear < 0.15:
        #       cv2.putText(frame, f'You blinked', (10, 30), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255, 255), 1)
             
    cv2.putText(frame, f'{int(fps)} fps', (580, 30), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255, 255), 1)   
    cv2.imshow('video', frame)
    if cv2.waitKey(1) == ord('q'):
        break
    

cam.release()
cv2.destroyAllWindows()   